<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/Array.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

Test Framework: C++ uses Boost.Test, Python will use unittest.
Python Implementation: The provided Python code uses the unittest module. This is evident from import unittest, the class definition class ArrayTests(unittest.TestCase):, and the use of unittest assertion methods like self.assertEqual(), self.assertTrue(), self.assertAlmostEqual(). The execution block if __name__ == '__main__': also employs unittest.TextTestRunner to run the tests.
Array Initialization: Python ql.Array has similar constructors. List comprehensions or direct list initialization are also common.
Python Implementation:
ql.Array() was used for the empty array (a1).
ql.Array(size) was used for a sized, default-initialized array (a2).
ql.Array(size, value) was used for an array filled with a constant value (a3).
ql.Array(size, value, increment) was used for an array with incremental values (a4).
ql.Array(a1) and ql.Array(a3) demonstrated initialization from another ql.Array (copying).
In test_array_operators, the get_array() function showed direct list initialization: return ql.Array([1.1, 2.2, 3.3]).
Iterators: Python ql.Array is iterable directly. The low-level iterator identity checks in C++ (like iter == a.begin()) are not straightforward or typically necessary to replicate in Python. We'll focus on the functional equivalence.
Python Implementation:
Iteration was primarily achieved using for i in range(len(a)): and accessing elements via a[i].
The C++ iterator identity checks in test_array_resize (e.g., BOOST_CHECK(iter == a.begin())) were deliberately not directly translated because Python's object identity for internal buffer pointers isn't typically exposed or tested this way. The Python tests focused on the functional outcome: verifying the array's size and that the values of elements were preserved or correctly initialized after resize.
L-value/R-value: This distinction is less explicit in Python. A function call returning an array effectively acts like an R-value.
Python Implementation: This was demonstrated in test_array_functions and test_array_operators.
An existing ql.Array variable a was used (analogous to an l-value).
The get_array() function, when called (e.g., ql.Pow(get_array(), exponential) or get_array() + get_array()), provided a temporary ql.Array object (analogous to an r-value) that was used directly in expressions.
std::transform: Python doesn't have a direct std::transform equivalent that modifies a ql.Array in-place with a functor in the same C++ STL style. We'll achieve this by iterating and assigning, or by creating a new array using list comprehensions. The C++ std::transform(a10.begin(), a10.end(), a10.begin(), FSquared()); modifies a10 in place, reading from a10 and writing to a10.
Python Implementation: In test_construction, the std::transform logic was replicated for a10 by:
Creating a temporary list of the original values from a10: original_values_a10 = [x for x in a10].
Iterating and assigning the transformed values back to a10, ensuring each transformation used the corresponding original value: a10[i] = f_squared(original_values_a10[i]). This correctly mimics the C++ behavior of reading from the original state of the array for each element's transformation.
Mathematical Functions: ql.Pow, ql.Exp, ql.Log, ql.Sqrt, ql.Abs are available. Also, numpy functions can be used if you convert the ql.Array to a numpy.array first, but it's better to use the QuantLib functions if they exist for ql.Array.
Python Implementation: The test_array_functions directly used the QuantLib-Python vectorized functions: ql.Pow(a, exponential), ql.Exp(a), ql.Log(a), ql.Sqrt(a), and ql.Abs(a). The results were then compared element-wise against Python's standard math module functions.
Error Checking: BOOST_ERROR and BOOST_FAIL map to self.fail(), BOOST_CHECK maps to self.assertTrue(), self.assertEqual(), etc. QL_CHECK_CLOSE maps to self.assertAlmostEqual().
Python Implementation:
self.assertTrue(a1.empty(), ...) was used for conditions expected to be true.
self.assertEqual(len(a2), size, ...) was used for exact value comparisons.
self.assertAlmostEqual(a4[i], value + i * increment, ...) was used for floating-point comparisons, analogous to QL_CHECK_CLOSE.
The custom helper self.assertArrayAlmostEqual was defined to encapsulate element-wise self.assertAlmostEqual for entire arrays, making the operator tests cleaner.
QL_EPSILON: This is available as ql.QL_EPSILON.
Python Implementation: ql.QL_EPSILON was imported and used to define the tolerance (tol or delta) for floating-point comparisons, for example: tol = 10 * ql.QL_EPSILON in test_array_functions.

In [ ]:
import QuantLib as ql
import unittest
import math

# Helper class equivalent to C++ FSquared
class FSquared:
    def __call__(self, x: float) -> float:
        return x * x

class ArrayTests(unittest.TestCase):

    def assertArrayAlmostEqual(self, arr1, arr2, delta, msg=""):
        self.assertEqual(len(arr1), len(arr2), f"{msg} Arrays have different lengths.")
        for i in range(len(arr1)):
            self.assertAlmostEqual(arr1[i], arr2[i], delta=delta,
                                   msg=f"{msg} Element {i} differs: {arr1[i]} vs {arr2[i]}")

    def test_construction(self):
        print("Testing array construction...")

        # empty array
        a1 = ql.Array()
        self.assertTrue(a1.empty(), f"default-initialized array is not empty (size = {len(a1)})")
        self.assertEqual(len(a1), 0)

        # sized array
        size = 5
        a2 = ql.Array(size)
        self.assertEqual(len(a2), size,
                         f"array not of the required size\n    required:  {size}\n    resulting: {len(a2)}")

        # sized array, constant values
        value = 42.0
        a3 = ql.Array(size, value)
        self.assertEqual(len(a3), size,
                         f"array not of the required size\n    required:  {size}\n    resulting: {len(a3)}")
        for i in range(size):
            self.assertEqual(a3[i], value,
                             f"{i+1}th element not with required value\n    required:  {value}\n    resulting: {a3[i]}")

        # sized array, incremental values
        increment = 3.0
        a4 = ql.Array(size, value, increment)
        self.assertEqual(len(a4), size,
                         f"array not of the required size\n    required:  {size}\n    resulting: {len(a4)}")
        for i in range(size):
            self.assertEqual(a4[i], value + i * increment,
                             f"{i+1}th element not with required value\n    required:  {value + i*increment}\n    resulting: {a4[i]}")

        # copy constructor / initialization from another array
        a5 = ql.Array(a1)
        self.assertEqual(len(a5), len(a1),
                         f"copy not of the same size as original\n    original:  {len(a1)}\n    copy:      {len(a5)}")

        a6 = ql.Array(a3)
        self.assertEqual(len(a6), len(a3),
                         f"copy not of the same size as original\n    original:  {len(a3)}\n    copy:      {len(a6)}")
        for i in range(len(a3)):
            self.assertEqual(a6[i], a3[i],
                             f"{i+1}th element of copy not with same value as original\n    original:  {a3[i]}\n    copy:      {a6[i]}")

        # transform
        a10 = ql.Array(5)
        for i in range(len(a10)):
            a10[i] = float(i)

        f_squared = FSquared()
        # In C++, std::transform(a10.begin(), a10.end(), a10.begin(), FSquared())
        # modifies a10 in place, applying FSquared to each original element.
        # We can simulate this by creating a list of transformed values and assigning back,
        # or by iterating and modifying.
        # To be absolutely sure we match the C++ behavior (reading original values):
        original_values_a10 = [x for x in a10]
        for i in range(len(a10)):
            a10[i] = f_squared(original_values_a10[i])

        for i in range(len(a10)):
            # The 'calculated' here refers to applying f_squared to the *original* i-th value, which was 'i'
            calculated = f_squared(float(i))
            self.assertAlmostEqual(a10[i], calculated, delta=1e-5,
                                   msg=f"Array transform test failed for element {i}: {a10[i]} vs {calculated}")


    def test_array_functions(self):
        print("Testing array functions...")

        def get_array():
            a = ql.Array(5)
            for i in range(len(a)):
                a[i] = math.sin(float(i)) + 1.1
            return a

        a = get_array()

        exponential = -2.3
        p_lvalue = ql.Pow(a, exponential)
        e_lvalue = ql.Exp(a)
        l_lvalue = ql.Log(a)
        s_lvalue = ql.Sqrt(a)
        a_lvalue = ql.Abs(a)

        p_rvalue = ql.Pow(get_array(), exponential)
        e_rvalue = ql.Exp(get_array())
        l_rvalue = ql.Log(get_array())
        s_rvalue = ql.Sqrt(get_array())
        a_rvalue = ql.Abs(get_array())

        tol = 10 * ql.QL_EPSILON
        for i in range(len(a)):
            self.assertAlmostEqual(p_lvalue[i], math.pow(a[i], exponential), delta=tol, msg="Array function test Pow failed (lvalue)")
            self.assertAlmostEqual(p_rvalue[i], math.pow(a[i], exponential), delta=tol, msg="Array function test Pow failed (rvalue)") # original C++ had (lvalue) here, assuming typo
            self.assertAlmostEqual(e_lvalue[i], math.exp(a[i]), delta=tol, msg="Array function test Exp failed (lvalue)")
            self.assertAlmostEqual(e_rvalue[i], math.exp(a[i]), delta=tol, msg="Array function test Exp failed (rvalue)")
            self.assertAlmostEqual(l_lvalue[i], math.log(a[i]), delta=tol, msg="Array function test Log failed (lvalue)")
            self.assertAlmostEqual(l_rvalue[i], math.log(a[i]), delta=tol, msg="Array function test Log failed (rvalue)")
            self.assertAlmostEqual(s_lvalue[i], math.sqrt(a[i]), delta=tol, msg="Array function test Sqrt failed (lvalue)")
            self.assertAlmostEqual(s_rvalue[i], math.sqrt(a[i]), delta=tol, msg="Array function test Sqrt failed (rvalue)")
            self.assertAlmostEqual(a_lvalue[i], abs(a[i]), delta=tol, msg="Array function test Abs failed (lvalue)") # math.fabs also works
            self.assertAlmostEqual(a_rvalue[i], abs(a[i]), delta=tol, msg="Array function test Abs failed (rvalue)")


    def test_array_resize(self):
        print("Testing array resize...")

        a = ql.Array(10, 1.0, 1.0) # start=1.0, increment=1.0

        for i in range(10):
            self.assertAlmostEqual(a[i], float(1 + i), delta=10 * ql.QL_EPSILON)

        a.resize(5)
        self.assertEqual(len(a), 5)

        for i in range(5):
            self.assertAlmostEqual(a[i], float(1 + i), delta=10 * ql.QL_EPSILON)

        # Store values before extending to check if they are preserved
        preserved_values = [a[i] for i in range(len(a))]

        a.resize(15)
        self.assertEqual(len(a), 15)

        for i in range(5): # Check preserved part
            self.assertAlmostEqual(a[i], float(1 + i), delta=10 * ql.QL_EPSILON)
            self.assertAlmostEqual(a[i], preserved_values[i], delta=10 * ql.QL_EPSILON)

        # The C++ test:
        # const Array::const_iterator iter = a.begin();
        # a.resize(a.size());
        # BOOST_CHECK(iter == a.begin());
        # This checks if the underlying data pointer remains the same if no reallocation
        # is needed. This is a low-level detail not easily or typically tested in Python.
        # We'll just check that resizing to the same size has no adverse effects on size.
        current_size = len(a)
        # first_element_id_before = id(a[0]) if current_size > 0 else None # This is not what C++ iter==a.begin() checks
        a.resize(current_size)
        self.assertEqual(len(a), current_size)
        # if current_size > 0:
        #     first_element_id_after = id(a[0])
            # self.assertEqual(first_element_id_before, first_element_id_after) # Again, not a direct translation

        # a.resize(10);
        # BOOST_CHECK(a.size() == 10);
        # BOOST_CHECK(iter == a.begin()); # Same as above, iterator identity check
        a.resize(10)
        self.assertEqual(len(a), 10)
        # if current_size > 0: # Original current_size was 15, now 10
        #    self.assertEqual(first_element_id_before, id(a[0])) # Still problematic for direct translation


    def test_array_operators(self):
        print("Testing array operators...")

        delta = 100 * ql.QL_EPSILON

        def get_array():
            # In Python, ql.Array can be initialized from a list
            return ql.Array([1.1, 2.2, 3.3])

        a = get_array()

        positive_expected = ql.Array([1.1, 2.2, 3.3])
        lvalue_positive = +a
        rvalue_positive = +get_array()

        self.assertArrayAlmostEqual(lvalue_positive, positive_expected, delta, "Unary + (lvalue)")
        self.assertArrayAlmostEqual(rvalue_positive, positive_expected, delta, "Unary + (rvalue)")

        negative_expected = ql.Array([-1.1, -2.2, -3.3])
        lvalue_negative = -a
        rvalue_negative = -get_array()

        self.assertArrayAlmostEqual(lvalue_negative, negative_expected, delta, "Unary - (lvalue)")
        self.assertArrayAlmostEqual(rvalue_negative, negative_expected, delta, "Unary - (rvalue)")

        array_sum_expected = ql.Array([2.2, 4.4, 6.6])
        lvalue_lvalue_sum = a + a
        lvalue_rvalue_sum = a + get_array()
        rvalue_lvalue_sum = get_array() + a
        rvalue_rvalue_sum = get_array() + get_array()

        self.assertArrayAlmostEqual(lvalue_lvalue_sum, array_sum_expected, delta, "Array + Array (l-l)")
        self.assertArrayAlmostEqual(lvalue_rvalue_sum, array_sum_expected, delta, "Array + Array (l-r)")
        self.assertArrayAlmostEqual(rvalue_lvalue_sum, array_sum_expected, delta, "Array + Array (r-l)")
        self.assertArrayAlmostEqual(rvalue_rvalue_sum, array_sum_expected, delta, "Array + Array (r-r)")

        scalar_sum_expected = ql.Array([2.2, 3.3, 4.4]) # 1.1 + original array
        lvalue_real_sum = a + 1.1
        rvalue_real_sum = get_array() + 1.1
        real_lvalue_sum = 1.1 + a
        real_rvalue_sum = 1.1 + get_array()

        self.assertArrayAlmostEqual(lvalue_real_sum, scalar_sum_expected, delta, "Array + Scalar (l-s)")
        self.assertArrayAlmostEqual(rvalue_real_sum, scalar_sum_expected, delta, "Array + Scalar (r-s)")
        self.assertArrayAlmostEqual(real_lvalue_sum, scalar_sum_expected, delta, "Scalar + Array (s-l)")
        self.assertArrayAlmostEqual(real_rvalue_sum, scalar_sum_expected, delta, "Scalar + Array (s-r)")

        array_difference_expected = ql.Array([0.0, 0.0, 0.0])
        lvalue_lvalue_difference = a - a
        lvalue_rvalue_difference = a - get_array()
        rvalue_lvalue_difference = get_array() - a
        rvalue_rvalue_difference = get_array() - get_array()

        self.assertArrayAlmostEqual(lvalue_lvalue_difference, array_difference_expected, delta, "Array - Array (l-l)")
        self.assertArrayAlmostEqual(lvalue_rvalue_difference, array_difference_expected, delta, "Array - Array (l-r)")
        self.assertArrayAlmostEqual(rvalue_lvalue_difference, array_difference_expected, delta, "Array - Array (r-l)")
        self.assertArrayAlmostEqual(rvalue_rvalue_difference, array_difference_expected, delta, "Array - Array (r-r)")

        scalar_difference_1_expected = ql.Array([0.0, 1.1, 2.2]) # a - 1.1
        scalar_difference_2_expected = ql.Array([0.0, -1.1, -2.2]) # 1.1 - a
        lvalue_real_difference = a - 1.1
        rvalue_real_difference = get_array() - 1.1
        real_lvalue_difference = 1.1 - a
        real_rvalue_difference = 1.1 - get_array()

        self.assertArrayAlmostEqual(lvalue_real_difference, scalar_difference_1_expected, delta, "Array - Scalar (l-s)")
        self.assertArrayAlmostEqual(rvalue_real_difference, scalar_difference_1_expected, delta, "Array - Scalar (r-s)")
        self.assertArrayAlmostEqual(real_lvalue_difference, scalar_difference_2_expected, delta, "Scalar - Array (s-l)")
        self.assertArrayAlmostEqual(real_rvalue_difference, scalar_difference_2_expected, delta, "Scalar - Array (s-r)")

        array_product_expected = ql.Array([1.1 * 1.1, 2.2 * 2.2, 3.3 * 3.3])
        lvalue_lvalue_product = a * a
        lvalue_rvalue_product = a * get_array()
        rvalue_lvalue_product = get_array() * a
        rvalue_rvalue_product = get_array() * get_array()

        self.assertArrayAlmostEqual(lvalue_lvalue_product, array_product_expected, delta, "Array * Array (l-l)")
        self.assertArrayAlmostEqual(lvalue_rvalue_product, array_product_expected, delta, "Array * Array (l-r)")
        self.assertArrayAlmostEqual(rvalue_lvalue_product, array_product_expected, delta, "Array * Array (r-l)")
        self.assertArrayAlmostEqual(rvalue_rvalue_product, array_product_expected, delta, "Array * Array (r-r)")

        scalar_product_expected = ql.Array([1.1 * 1.1, 2.2 * 1.1, 3.3 * 1.1]) # a * 1.1
        lvalue_real_product = a * 1.1
        rvalue_real_product = get_array() * 1.1
        real_lvalue_product = 1.1 * a
        real_rvalue_product = 1.1 * get_array()

        self.assertArrayAlmostEqual(lvalue_real_product, scalar_product_expected, delta, "Array * Scalar (l-s)")
        self.assertArrayAlmostEqual(rvalue_real_product, scalar_product_expected, delta, "Array * Scalar (r-s)")
        self.assertArrayAlmostEqual(real_lvalue_product, scalar_product_expected, delta, "Scalar * Array (s-l)")
        self.assertArrayAlmostEqual(real_rvalue_product, scalar_product_expected, delta, "Scalar * Array (s-r)")

        array_quotient_expected = ql.Array([1.0, 1.0, 1.0])
        lvalue_lvalue_quotient = a / a
        lvalue_rvalue_quotient = a / get_array()
        rvalue_lvalue_quotient = get_array() / a
        rvalue_rvalue_quotient = get_array() / get_array()

        self.assertArrayAlmostEqual(lvalue_lvalue_quotient, array_quotient_expected, delta, "Array / Array (l-l)")
        self.assertArrayAlmostEqual(lvalue_rvalue_quotient, array_quotient_expected, delta, "Array / Array (l-r)")
        self.assertArrayAlmostEqual(rvalue_lvalue_quotient, array_quotient_expected, delta, "Array / Array (r-l)")
        self.assertArrayAlmostEqual(rvalue_rvalue_quotient, array_quotient_expected, delta, "Array / Array (r-r)")

        scalar_quotient_1_expected = ql.Array([1.1 / 1.1, 2.2 / 1.1, 3.3 / 1.1]) # a / 1.1
        scalar_quotient_2_expected = ql.Array([1.1 / 1.1, 1.1 / 2.2, 1.1 / 3.3]) # 1.1 / a
        lvalue_real_quotient = a / 1.1
        rvalue_real_quotient = get_array() / 1.1
        real_lvalue_quotient = 1.1 / a
        real_rvalue_quotient = 1.1 / get_array()

        self.assertArrayAlmostEqual(lvalue_real_quotient, scalar_quotient_1_expected, delta, "Array / Scalar (l-s)")
        self.assertArrayAlmostEqual(rvalue_real_quotient, scalar_quotient_1_expected, delta, "Array / Scalar (r-s)")
        self.assertArrayAlmostEqual(real_lvalue_quotient, scalar_quotient_2_expected, delta, "Scalar / Array (s-l)")
        self.assertArrayAlmostEqual(real_rvalue_quotient, scalar_quotient_2_expected, delta, "Scalar / Array (s-r)")


if __name__ == '__main__':
    print("Python QuantLib version: " + ql.__version__)
    suite = unittest.TestSuite()
    suite.addTest(unittest.makeSuite(ArrayTests))
    unittest.TextTestRunner(verbosity=2).run(suite)